In [94]:
import psycopg2
import pandas as pd

In [95]:
def create_database():
    # Connect to default database for init
    # conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    # conn.set_session(autocommit=True)
    # cur = conn.cursor()

    # Create superstore database
    # cur.execute("DROP DATABASE superstore")
    # cur.execute("CREATE DATABASE superstore")

    # Close default database connection
    # conn.close()

    # Connect to superstore database
    conn = psycopg2.connect("host=127.0.0.1 dbname=superstore user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

In [96]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [97]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

### Dataset preprocessing
Dataset is an excel file containing the sales transaction data from a retail store. There are 3 separate sheets in the file namely orders, people, and returns. The goal of this preprocessing is to separate the 3 sheets into distinct dataset and normalize the orders dataset to which we will use star schema to build the data model.

In [98]:
# Load data
superstore_orders = pd.read_excel("dataset\superstore.xls", sheet_name="Orders")
superstore_people = pd.read_excel("dataset\superstore.xls", sheet_name="People")
superstore_returns = pd.read_excel("dataset\superstore.xls", sheet_name="Returns")

In [99]:
superstore_orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [100]:
superstore_orders.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [101]:
superstore_people.head()


,Person,Region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


In [102]:
superstore_returns.head()

,Returned,Order ID
0,Yes,CA-2017-153822
1,Yes,CA-2017-129707
2,Yes,CA-2014-152345
3,Yes,CA-2015-156440
4,Yes,US-2017-155999


In [103]:
# Check for duplcate rows (No duplicate found)
print("Order dataset duplicate found:", superstore_orders.duplicated().sum())
print("Return dataset duplicate found:", superstore_returns.duplicated().sum())
print("People dataset duplicate found:", superstore_people.duplicated().sum())

# Check for missing values (No missing value found)
print("Order dataset missing value found:", superstore_orders.isnull().sum())
print("Return dataset missing value found:", superstore_returns.isnull().sum())
print("People dataset missing value found:", superstore_people.isnull().sum())

Order dataset duplicate found: 0
Return dataset duplicate found: 0
People dataset duplicate found: 0
Order dataset missing value found: Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64
Return dataset missing value found: Returned    0
Order ID    0
dtype: int64
People dataset missing value found: Person    0
Region    0
dtype: int64


In [104]:
# Normalizing main table
stg_customer_table  = superstore_orders[['Customer ID', 'Customer Name', 'Segment']]
stg_location_table  = superstore_orders[['Country', 'Customer ID', 'City', 'State', 'Postal Code', 'Region']]
stg_order_table     = superstore_orders[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID']]
stg_product_table   = superstore_orders[['Product ID', 'Category', 'Sub-Category', 'Product Name']]
stg_sales_table     = superstore_orders[['Row ID', 'Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit']]

# Add primary key for tables without them
stg_location_table.insert(0, 'Location_ID', range(1, len(stg_location_table) + 1))

In [105]:
# Drop duplicates
stg_customer_table = stg_customer_table.drop_duplicates()
stg_location_table = stg_location_table.drop_duplicates()
stg_order_table = stg_order_table.drop_duplicates()
stg_product_table = stg_product_table.drop_duplicates()
stg_sales_table = stg_sales_table.drop_duplicates()

In [106]:
# Renaming columns
stg_customer_table = stg_customer_table.rename(columns={
    'Customer ID': 'customer_id',
    'Customer Name': 'customer_name',
    'Segment': 'customer_segment'
})

stg_location_table = stg_location_table.rename(columns={
    'Customer ID' : 'customer_id',
    'Location_ID' : 'location_id',
    'Country' : 'country',
    'City' : 'city',
    'State' : 'state',
    'Postal Code' : 'postal_code', 
    'Region' : 'region'
})

stg_order_table = stg_order_table.rename(columns={
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Ship Date': 'ship_date',
    'Ship Mode': 'ship_mode',
    'Customer ID': 'customer_id'
})

stg_product_table = stg_product_table.rename(columns={
    'Product ID': 'product_id',
    'Category': 'product_category',
    'Sub-Category': 'product_subcategory',
    'Product Name': 'product_name'
})

stg_sales_table = stg_sales_table.rename(columns={
    'Row ID': 'sales_row_id',
    'Order ID': 'order_id', 
    'Product ID': 'product_id', 
    'Sales': 'sales_amount',
    'Quantity': 'sales_quantity',
    'Discount': 'sales_discount',
    'Profit': 'sales_profit'
})

In [107]:
cur, conn = create_database()

In [108]:
customer_table_creation = (
                           """ 
                           CREATE TABLE IF NOT EXISTS customers (
                           customer_id VARCHAR PRIMARY KEY,
                           customer_name VARCHAR,
                           customer_segment VARCHAR)
                           """
                           )

location_table_creation = (
                           """ 
                           CREATE TABLE IF NOT EXISTS locations (
                           location_id INT PRIMARY KEY,
                           country VARCHAR,
                           customer_id VARCHAR,
                           city VARCHAR,
                           state VARCHAR,
                           postal_code INT,
                           region VARCHAR)
                           """
                           )

order_table_creation = (
                        """ 
                        CREATE TABLE IF NOT EXISTS orders (
                        order_id VARCHAR PRIMARY KEY,
                        order_date DATE,
                        ship_date DATE,
                        ship_mode VARCHAR,
                        customer_id VARCHAR)
                        """
                        )

product_table_creation = (
                        """ 
                        CREATE TABLE IF NOT EXISTS products (
                        product_id VARCHAR PRIMARY KEY,
                        product_category VARCHAR,
                        product_subcategory VARCHAR,
                        product_name VARCHAR)
                        """
                        )

sales_table_creation = (
                        """ 
                        CREATE TABLE IF NOT EXISTS sales (
                        sales_row_id INT PRIMARY KEY,
                        order_id VARCHAR,
                        product_id VARCHAR,
                        sales_amount NUMERIC,
                        sales_quantity INT,
                        sales_discount NUMERIC,
                        sales_profit NUMERIC)
                        """
                        )

In [109]:
# Generate tables in Postgres
cur.execute(customer_table_creation)
conn.commit()
cur.execute(location_table_creation)
conn.commit()
cur.execute(order_table_creation)
conn.commit()
cur.execute(product_table_creation)
conn.commit()
cur.execute(sales_table_creation)
conn.commit()

In [110]:
customer_table_data_insert =  """
                            INSERT INTO customers (
                                customer_id,
                                customer_name,
                                customer_segment
                            ) VALUES (%s, %s, %s)
                            """

location_table_data_insert = """ 
                           INSERT INTO locations (
                           location_id,
                           country,
                           customer_id,
                           city,
                           state,
                           postal_code,
                           region
                           ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                           """

order_table_data_insert = """ 
                        INSERT INTO orders (
                        order_id,
                        order_date,
                        ship_date,
                        ship_mode,
                        customer_id
                        ) VALUES (%s, %s, %s, %s, %s)
                        """

product_table_data_insert = """ 
                        INSERT INTO products (
                        product_id,
                        product_category,
                        product_subcategory,
                        product_name
                        ) VALUES (%s, %s, %s, %s)
                        """

sales_table_data_insert = """ 
                        INSERT INTO sales (
                        sales_row_id,
                        order_id,
                        product_id,
                        sales_amount,
                        sales_quantity,
                        sales_discount,
                        sales_profit
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                        """

In [ ]:
for i, row in stg_customer_table.iterrows():
    cur.execute(customer_table_data_insert, list(row))


In [ ]:
for i, row in stg_location_table.iterrows():
    cur.execute(location_table_data_insert, list(row))

In [ ]:
for i, row in stg_order_table.iterrows():
    cur.execute(order_table_data_insert, list(row))

In [ ]:
for i, row in stg_product_table.iterrows():
    cur.execute(product_table_data_insert, list(row))

In [123]:
for i, row in stg_sales_table.iterrows():
    cur.execute(sales_table_data_insert, list(row))

In [125]:
cur.close()
conn.close()